<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#A-simulation-with-WOFOST" data-toc-modified-id="A-simulation-with-WOFOST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>A simulation with WOFOST</a></span><ul class="toc-item"><li><span><a href="#Your-first-WOFOST-run" data-toc-modified-id="Your-first-WOFOST-run-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Your first WOFOST run</a></span></li></ul></li><li><span><a href="#Some-important-WOFOST-parameters" data-toc-modified-id="Some-important-WOFOST-parameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Some important WOFOST parameters</a></span></li></ul></div>

<div style="float:right">
<table>
<tr>
    <td> <img src="figs/nceo_logo.png" alt="Drawing" style="width:200px;height:40px;"/> </td>
    <td> <img src="figs/gssti_logo.png" alt="Drawing" style="width:200px;height:40px;"/> </td>
    <td> <img src="figs/multiply_logo.png" alt="Drawing" style="width:40px;height:40px;"/> </td>
</tr>
    </table>
</div>

# The WOFOST crop model

**Author:** Jose Gómez-Dans (NCEO & UCL)  `j.gomez-dans@ucl.ac.uk`

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os
import copy
import datetime as dt
import pcse
from pcse.models import Wofost71_WLP_FD
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)
import datetime

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from wofost_utils import *

This notebook was built with:
python version: 3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0] 
PCSE version: 5.4.2


## Introduction

The WOFOST crop model has been around for a long time. It is widely used in both research and operational applications, and is fairly easy to use. It also has been used to simulate a large number of crops, and there are a number of extensions that deal with other crops.

While WOFOST lacks some of the more complex processes that more sophisticated models provide, it's more than adequate to familiarise oneself with a crop model.

The following diagram shows what WOFOST does

![schematic of the WOFOST model](figs/wofost_schematic.png)

Some of those boxes will already be familiar to you. We can see at the top that WOFOST is driven by radiation intercepted by the canopy. The canopy interception itself is not driven by a satellite fAPAR product as we saw earlier, but rather uses the predictions of LAI from the previous time step. Once photosynthesis is calculated and both growth and maintenance respiration are lost, partitioning of the assimilated C to the different plant organs takes place. These organs are:

1. Leaves
2. Stems
3. Roots
4. Storage organs (e.g. grain!)

A number of processes deal with the age and eventual death of leaves, stems and roots.

WOFOST implements a large number of processes to model a crop. Although not shown here, it does have a soil moisture module, and many of the processes are controlled by the development stage of the crop (its *phenology*). The phenology is simulated often by thermal time (there are other options, using e.g. daylength and stuff).

WOFOST doesn't implement many processes that are to do with management, pests, complex soil chemistry, etc. However, some management and additional information is needed to run the model, such as selecting the crop and variety, providing meteo data, configuring things around... This is all done for you (lucky you!). Since our aim here is to build towards linking the model with EO data, we'll consider some parameters that are important to diagnose what's happening. These will include LAI, soil moisture, total storage organ weight, and the development stage. 


## A simulation with WOFOST

There are two ways of using the model: potential and water limited. the former ignores the effect of water stress, whereas the latter does some soil water calculations and includes the effect of soil moisture in crop development. 

A simulation starts by the user selecting a soil type, a crop variety, some meteorological inputs, and some other parameters such as either the sowing or emergence date, the amount of water in the soil at the start of the simulation, etc. Once this is done, the model takes over, calculating soil hydraulics and any limitations to growth due to lack of soil water, and calculating radiation interception, photosynthesis, removing respiration losses and eventually partioning the assimilate. Defined by two crop variety parameters, the model calculates the thermal time between emergence and flowering, and between flowering and harvest, this is the development stage (DVS), which goes from 0 (emergence) to 1 (flowering) to 2 (harvest). Many processes are impacted by DVS.

### Your first WOFOST run

The next grid will run WOFOST and plot some graphs of some state variables with respect to time as a function of sowing and harvest date. The code will run the simulation changing the sowing date from sowing to harvest date, every `n_days`. You can select different regions and/or years.

In [2]:
change_sowing_slider();

interactive(children=(DatePicker(value=datetime.date(2011, 7, 1), description='Earliest possible sowing date')…

You can probably see some relationships emerging between some parameters. You can also see that changing the sowing date results in massively different trajectories for LAI and biomass allocation to the different organs, but that ultimately, the variation in yield seems to be quite small. A reason for this is that once the crop reaches maturity (DVS=2), there's not assimilation going on. Also, as the leaves age and senesce, there's isn't that much C going into the system, which produces a saturation pattern.

The example above is a gross simplification. We have given you the ability to change the meteorological inputs, but we still use the same crop parameters, same soil properties and same starting values for everything. Changing these settings would results in different trajectories. Let's have a play with some important parameters

## Some important WOFOST parameters

WOFOST has got tons of parameters, and it would be unwise to just list all of them. We will play around with just a handful to see the effect they have on crop behaviour. Here's a list of them.

* `WAV`: Initial amount of available water in total rootable zone (moisture content above wilting point) (cm)
* `RDMSOL`: Maximum rooting depth allowed by the soil
* `SPAN`: life span of leaves growing at an average temperature of 35 °C
* `TDWI`: Initial total dry weight of crop
* `RGRLAI`: Relative growth in LAI per day
* `TSUMEM`: Length (in effective degree days) from sowing to emergence 
* `TSUM1`: Length (in effective degree days) of the vegetative period
* `TSUM2`: Length (in effective degree days) of the reproductive period


`TSUM1` and `TSUM2` are critical in controlling phenology

In [14]:
wofost_parameter_sweep();

interactive(children=(FloatSlider(value=40.0, description='span', max=50.0, min=20.0), FloatSlider(value=20.0,…